<a href="https://colab.research.google.com/github/juanjo101/-Dashboard-Financiero/blob/main/Comparador_Financiero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# COMPARADOR DE EMPRESAS (ADDON INDEPENDIENTE) — 3 empresas
# No modifica tu script original
# ============================================
!pip -q install gradio plotly openpyxl

import re, numpy as np, pandas as pd
import plotly.graph_objects as go
import gradio as gr

pd.options.display.float_format = '{:,.2f}'.format

# ---------- Utilidades (copias simplificadas del original) ----------
def _to_number(x):
    if pd.isna(x): return np.nan
    if isinstance(x, (int, float, np.number)): return float(x)
    s = str(x).strip().replace(' ', '').replace('%','')
    if s.count(',') > 1 and '.' in s:
        s = s.replace(',', '')
    elif s.count('.') > 1 and ',' in s:
        s = s.replace('.', '').replace(',', '.')
    else:
        if ',' in s and '.' not in s:
            s = s.replace(',', '.')
        s = s.replace(',', '')
    try:
        return float(s)
    except:
        return np.nan

def parse_multi_year_sheet(path, sheet_name):
    df = pd.read_excel(path, sheet_name=sheet_name, header=None)
    hdr = None
    for r in range(min(50, df.shape[0])):
        if str(df.iat[r, 0]).strip().upper() == 'CONCEPTOS':
            hdr = r
            break
    if hdr is None:
        raise ValueError(f"No se encontró 'CONCEPTOS' en la hoja {sheet_name}")
    date_row = hdr + 1

    def _get_year_from(cell):
        dt = pd.to_datetime(cell, errors='coerce')
        if pd.isna(dt): return None
        return int(dt.year)

    amount_cols, year_labels = [], []
    for c in range(1, df.shape[1]):
        top = df.iat[hdr, c]
        below = df.iat[date_row, c] if date_row < df.shape[0] else None
        y = None
        if isinstance(top, str) and top.strip().upper() == 'FECHA':
            y = _get_year_from(below)
        else:
            y = _get_year_from(top)
        if y is not None:
            amount_cols.append(c)
            year_labels.append(str(y))
    if not amount_cols:
        raise ValueError(f"No se detectaron columnas de año en {sheet_name}")

    start = hdr + 2
    sub = df.iloc[start:, [0] + amount_cols].copy()
    temp_cols = ['Cuenta'] + [f"Y{i}_{y}" for i, y in enumerate(year_labels)]
    sub.columns = temp_cols
    sub['Cuenta'] = sub['Cuenta'].astype(str).str.strip()
    for c in temp_cols[1:]:
        sub[c] = sub[c].apply(_to_number)

    uniq_years = sorted(set(year_labels))
    out = pd.DataFrame({'Cuenta': sub['Cuenta']})
    for y in uniq_years:
        cols_y = [c for c in temp_cols[1:] if c.endswith(f"_{y}")]
        out[y] = sub[cols_y].sum(axis=1, skipna=True)
    out = out.dropna(how='all', subset=uniq_years).reset_index(drop=True)
    return out, uniq_years

def vertical_analysis_exact(df, year, total_patterns, prefer_max=True):
    out = df[['Cuenta', year]].copy()
    mask = False
    for pat in total_patterns:
        mask = mask | out['Cuenta'].str.contains(pat, case=False, regex=True, na=False)
    matches = out.loc[mask, year]
    total = (matches.max(skipna=True) if (not matches.empty and prefer_max) else
             matches.sum(skipna=True) if not matches.empty else out[year].sum(skipna=True))
    out[f'%_{year}'] = np.where(total == 0, np.nan, out[year] / total * 100.0)
    return out, total

def horizontal_analysis(df, years_sorted):
    out = df[['Cuenta'] + years_sorted].copy()
    for i in range(1, len(years_sorted)):
        a, b = years_sorted[i-1], years_sorted[i]
        out[f'Var% {a}->{b}'] = np.where(out[a].fillna(0)==0, np.nan, (out[b]-out[a])/out[a]*100.0)
    return out

def pick_value(df, patterns, year, prefer_max=True):
    mask = pd.Series(False, index=df.index)
    for pat in patterns:
        looks_regex = bool(re.search(r'[.\^\$\*\+\?\{\}\[\]\|\(\)\\]', pat))
        if looks_regex:
            pat_nc = re.sub(r'\((?!\?)', '(?:', pat)
            m = df['Cuenta'].str.contains(pat_nc, case=False, regex=True, na=False)
        else:
            m = df['Cuenta'].str.contains(pat, case=False, regex=False, na=False)
        mask = mask | m
    vals = df.loc[mask, year]
    if vals.empty: return np.nan
    return vals.max(skipna=True) if prefer_max else vals.sum(skipna=True)

def sdiv(n, d):
    return np.nan if (d in [0, None] or pd.isna(d) or pd.isna(n) or d == 0) else n/d

def build_ratios(balance_wide, eres_wide, bal_years, er_years):
    years_common = sorted(set(bal_years).intersection(set(er_years)))
    ratios = pd.DataFrame(index=[int(y) for y in years_common])
    for y in years_common:
        AT  = pick_value(balance_wide, [r'^TOTAL\s+ACTIVOS?$'], y)
        PT  = pick_value(balance_wide, [r'^TOTAL\s+PASIVOS?$'], y)
        PAT = pick_value(balance_wide, [r'RECURSOS\s+PROPIOS', r'^PATRIMONIO$', r'CAPITAL\s+CONTABLE', r'FONDOS\s+PROPIOS'], y)
        AC  = pick_value(balance_wide, [r'^ACTIVO\s+(CIRCULANTE|CORRIENTE)$'], y)
        PC  = pick_value(balance_wide, [r'^PASIVO\s+(CIRCULANTE|CORRIENTE)$'], y)
        INV = pick_value(balance_wide, [r'INVENTARIOS?$', r'^EXISTENCIAS$'], y, prefer_max=False)
        VN  = pick_value(eres_wide, [r'^\s*\.?\s*=\s*INGRESOS\s+TOTALES\s*$', r'^\s*VENTAS\s*$', r'^\s*INGRESOS\s+NETOS\s*$'], y)
        UN  = pick_value(eres_wide, [r'UTILIDAD\s+NETA$', r'BENEFICIO\s+NETO$', r'GANANCIA\s+NETA$'], y)
        ratios.loc[int(y), 'Liquidez Corriente']                  = sdiv(AC, PC)
        ratios.loc[int(y), 'Prueba Ácida']                        = sdiv((AC - (0 if pd.isna(INV) else INV)), PC)
        ratios.loc[int(y), 'Endeudamiento (Pasivo/Activo)']       = sdiv(PT, AT)
        ratios.loc[int(y), 'Apalancamiento (Activo/Patrimonio)']  = sdiv(AT, PAT)
        ratios.loc[int(y), 'Margen Neto']                         = sdiv(UN, VN)
        ratios.loc[int(y), 'ROA']                                 = sdiv(UN, AT)
        ratios.loc[int(y), 'ROE']                                 = sdiv(UN, PAT)
        ratios.loc[int(y), 'Rotación de Activos']                 = sdiv(VN, AT)
    ratios.index.name = 'Año'
    return ratios

def kpis_core(balance_wide, eres_wide, years):
    years = [int(y) for y in years]
    series = {}
    for label, pats in {
        'Ventas': [r'^\s*\.?\s*=\s*INGRESOS\s+TOTALES\s*$', r'^\s*VENTAS\s*$', r'^\s*INGRESOS\s+NETOS\s*$'],
        'Utilidad Neta': [r'UTILIDAD\s+NETA$', r'BENEFICIO\s+NETO$', r'GANANCIA\s+NETA$'],
        'Total Activos': [r'^TOTAL\s+ACTIVOS?$'],
        'Total Pasivos': [r'^TOTAL\s+PASIVOS?$'],
        'Patrimonio':    [r'RECURSOS\s+PROPIOS', r'^PATRIMONIO$', r'CAPITAL\s+CONTABLE', r'FONDOS\s+PROPIOS'],
    }.items():
        vals = {}
        for y in years:
            df = eres_wide if label in ['Ventas','Utilidad Neta'] else balance_wide
            vals[y] = pick_value(df, pats, str(y))
        series[label] = pd.Series(vals, name=label).sort_index()
    return series

# ---------- Pipeline por empresa ----------
def compute_company_metrics(xlsx_path):
    bal_w, bal_years = parse_multi_year_sheet(xlsx_path, 'ESTRUCTURA FINANCIERA')
    er_w,  er_years  = parse_multi_year_sheet(xlsx_path, 'ESTRUCTURA ECONOMICA')

    bal_vertical = {y: vertical_analysis_exact(bal_w, y, [r'^TOTAL\s+ACTIVOS?$'])[0] for y in bal_years}
    er_vertical  = {y: vertical_analysis_exact(er_w,  y,
                       [r'^\s*\.?\s*=\s*INGRESOS\s+TOTALES\s*$', r'^\s*VENTAS\s*$', r'^\s*INGRESOS\s+NETOS\s*$'])[0] for y in er_years}

    bal_h = horizontal_analysis(bal_w, bal_years)
    er_h  = horizontal_analysis(er_w,  er_years)

    ratios = build_ratios(bal_w, er_w, bal_years, er_years)
    kpis   = kpis_core(bal_w, er_w, sorted(set(bal_years).union(er_years)))

    return {
        "balance_wide": bal_w, "er_wide": er_w,
        "bal_years": bal_years, "er_years": er_years,
        "bal_vertical": bal_vertical, "er_vertical": er_vertical,
        "bal_h": bal_h, "er_h": er_h,
        "ratios": ratios, "kpis": kpis
    }

# ---------- Helpers multi-empresa ----------
def _legend_name_from_path(path):
    return (path or "").split("/")[-1]

def _ratios_intersection(emps):
    cols_sets = []
    for _, m in emps.items():
        cols_sets.append(set(m["ratios"].columns))
    if not cols_sets:
        return []
    inter = cols_sets[0].copy()
    for s in cols_sets[1:]:
        inter &= s
    return sorted(list(inter), key=lambda x: ["Liquidez Corriente","Prueba Ácida",
                                              "Endeudamiento (Pasivo/Activo)","Apalancamiento (Activo/Patrimonio)",
                                              "Margen Neto","ROA","ROE","Rotación de Activos"].index(x)
                  if x in ["Liquidez Corriente","Prueba Ácida","Endeudamiento (Pasivo/Activo)",
                           "Apalancamiento (Activo/Patrimonio)","Margen Neto","ROA","ROE","Rotación de Activos"] else 999)

def _years_union_from_ratios(emps):
    yrs = set()
    for _, m in emps.items():
        yrs |= set(map(int, m["ratios"].index))
    return sorted(yrs)

def _vyears_union(emps):
    yrs = set()
    for _, m in emps.items():
        yrs |= set(map(int, m["bal_vertical"].keys()))
        yrs |= set(map(int, m["er_vertical"].keys()))
    return sorted(yrs)

# ---------- Callbacks ----------
def _load(a, b, c):
    if a is None or b is None:
        raise gr.Error("Sube al menos los archivos de **Empresa A** y **Empresa B** (.xlsx). La Empresa C es opcional.")

    emps = {}
    names = []

    a_nm = _legend_name_from_path(a.name)
    b_nm = _legend_name_from_path(b.name)
    emps[a_nm] = compute_company_metrics(a.name)
    emps[b_nm] = compute_company_metrics(b.name)
    names.extend([a_nm, b_nm])

    if c is not None:
        c_nm = _legend_name_from_path(c.name)
        emps[c_nm] = compute_company_metrics(c.name)
        names.append(c_nm)

    ratio_names = _ratios_intersection(emps) or sorted(next(iter(emps.values()))["ratios"].columns.tolist())
    yrs  = _years_union_from_ratios(emps)
    vyrs = _vyears_union(emps)

    # Empresa por defecto para vertical
    empresa_default = names[0] if names else None
    ratio_default   = ratio_names[0] if ratio_names else None
    year_default    = vyrs[-1] if vyrs else None

    return (
        emps,                 # EMPS (dict)
        names,                # NAMES (list)
        yrs,                  # YEARS (list)
        gr.update(choices=ratio_names, value=ratio_default),   # ratio_dd
        gr.update(choices=vyrs, value=year_default),           # year_dd
        gr.update(choices=names, value=empresa_default)        # empresa_sel_dd
    )

def _plot_kpi_multi(kpi, EMPS, YEARS, NAMES):
    if not EMPS or not kpi:
        return go.Figure(), pd.DataFrame()
    comb = pd.DataFrame({"Año": YEARS})
    fig = go.Figure()
    for nm in NAMES:
        s = EMPS[nm]["kpis"][kpi].copy()
        comb[nm] = comb["Año"].map(s.to_dict())
        fig.add_trace(go.Scatter(x=comb["Año"].astype(str), y=comb[nm], mode="lines+markers", name=nm))
    fig.update_layout(title=f"{kpi} — Comparado", xaxis_title="Año", yaxis_title="Monto", template="plotly_white")
    return fig, comb

def _plot_ratio_multi(rname, EMPS, NAMES):
    if not EMPS or not rname:
        return go.Figure(), pd.DataFrame()
    # años unión para esa métrica
    yrs = set()
    for nm in NAMES:
        r = EMPS[nm]["ratios"]
        if rname in r.columns:
            yrs |= set(map(int, r.index))
    yrs = sorted(yrs)
    comb = pd.DataFrame({"Año": yrs})
    fig = go.Figure()
    for nm in NAMES:
        r = EMPS[nm]["ratios"]
        vals = r[rname].to_dict() if rname in r.columns else {}
        comb[nm] = comb["Año"].map(vals)
        fig.add_trace(go.Scatter(x=comb["Año"].astype(str), y=comb[nm], mode="lines+markers", name=nm))
    fig.update_layout(title=f"{rname} — Comparado", xaxis_title="Año", yaxis_title="Valor", template="plotly_white")
    return fig, comb

def _horizontal_tables_multi(EMPS, NAMES):
    if not EMPS:
        return pd.DataFrame(), pd.DataFrame()
    # Alinear por 'Cuenta' y unir columnas para todas las empresas presentes
    def _align_common_cols(tables):
        # buscar columnas comunes (por años y Var%)
        common = set(tables[0].columns)
        for t in tables[1:]:
            common &= set(t.columns)
        common = ['Cuenta'] + [c for c in sorted(common) if c != 'Cuenta']
        return common

    # Balance horizontal
    bal_tables = [EMPS[nm]["bal_h"].copy() for nm in NAMES]
    if len(bal_tables) >= 2:
        common_cols = _align_common_cols(bal_tables)
        for i in range(len(bal_tables)):
            bal_tables[i] = bal_tables[i][common_cols].set_index('Cuenta')
        bal_comb = pd.concat({nm: bal_tables[i] for i, nm in enumerate(NAMES)}, axis=1).reset_index()
        bal_comb.columns = ['Cuenta'] + [f"{lvl1}_{lvl2}" for lvl1, lvl2 in bal_comb.columns.to_list()[1:]]
    else:
        bal_comb = pd.DataFrame()

    # ER horizontal
    er_tables = [EMPS[nm]["er_h"].copy() for nm in NAMES]
    if len(er_tables) >= 2:
        common_cols_er = _align_common_cols(er_tables)
        for i in range(len(er_tables)):
            er_tables[i] = er_tables[i][common_cols_er].set_index('Cuenta')
        er_comb = pd.concat({nm: er_tables[i] for i, nm in enumerate(NAMES)}, axis=1).reset_index()
        er_comb.columns = ['Cuenta'] + [f"{lvl1}_{lvl2}" for lvl1, lvl2 in er_comb.columns.to_list()[1:]]
    else:
        er_comb = pd.DataFrame()

    return bal_comb, er_comb

def _vertical_by_year_select(empresa_sel, y, EMPS, NAMES):
    # Grafico solo de la empresa seleccionada; tablas de A/B/C (si existen)
    if not EMPS or not empresa_sel or y is None:
        return go.Figure(), go.Figure(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    y = str(int(y))

    def _plot_top(df):
        if df is None or df.empty or f"%_{y}" not in df.columns:
            return go.Figure()
        tmp = df[['Cuenta', f"%_{y}"]].dropna()
        tmp = tmp[~tmp['Cuenta'].astype(str).str.contains('TOTAL', case=False, na=False)]
        tmp = tmp.sort_values(f"%_{y}", ascending=False).head(10)
        fig = go.Figure(go.Bar(x=tmp[f"%_{y}"].values, y=tmp['Cuenta'].values, orientation='h'))
        fig.update_layout(title=f"Top 10 % ({y}) — {empresa_sel}", xaxis_title="Porcentaje (%)", yaxis_title="", template="plotly_white")
        return fig

    # Plots de la empresa seleccionada
    vbal_sel = EMPS[empresa_sel]["bal_vertical"].get(y, pd.DataFrame())
    ver_sel  = EMPS[empresa_sel]["er_vertical"].get(y,  pd.DataFrame())
    plot_bal = _plot_top(vbal_sel)
    plot_er  = _plot_top(ver_sel)

    # Tablas para A/B/C en orden NAMES
    def _get_or_empty(nm, which):
        if which == "bal":
            return EMPS[nm]["bal_vertical"].get(y, pd.DataFrame())
        else:
            return EMPS[nm]["er_vertical"].get(y, pd.DataFrame())

    tbl_bal_A = _get_or_empty(NAMES[0], "bal") if len(NAMES) >= 1 else pd.DataFrame()
    tbl_bal_B = _get_or_empty(NAMES[1], "bal") if len(NAMES) >= 2 else pd.DataFrame()
    tbl_bal_C = _get_or_empty(NAMES[2], "bal") if len(NAMES) >= 3 else pd.DataFrame()

    tbl_er_A = _get_or_empty(NAMES[0], "er") if len(NAMES) >= 1 else pd.DataFrame()
    tbl_er_B = _get_or_empty(NAMES[1], "er") if len(NAMES) >= 2 else pd.DataFrame()
    tbl_er_C = _get_or_empty(NAMES[2], "er") if len(NAMES) >= 3 else pd.DataFrame()

    # Devuelvo plots + 6 tablas (A/B/C para Balance y ER)
    return plot_bal, plot_er, tbl_bal_A, tbl_bal_B, tbl_bal_C, tbl_er_A, tbl_er_B, tbl_er_C

def _exports_multi(EMPS, NAMES):
    if not EMPS:
        return None, None

    # Ratios combinados
    all_ratio_names = sorted(set().union(*[set(EMPS[nm]["ratios"].columns) for nm in NAMES]))
    all_years_rat = sorted(set().union(*[set(map(int, EMPS[nm]["ratios"].index)) for nm in NAMES]))
    rat = pd.DataFrame({"Año": all_years_rat})
    for nm in NAMES:
        for r in all_ratio_names:
            vals = EMPS[nm]["ratios"][r].to_dict() if r in EMPS[nm]["ratios"].columns else {}
            rat[f"{r}__{nm}"] = rat["Año"].map(vals)
    rat_path = "/content/ratios_comparados_ABC.csv"
    rat.to_csv(rat_path, index=False)

    # KPIs combinados
    kpi_names = ['Ventas','Utilidad Neta','Total Activos','Total Pasivos','Patrimonio']
    all_years_kpi = sorted(set().union(*[set(EMPS[nm]["kpis"]['Ventas'].index) for nm in NAMES]))
    kdf = pd.DataFrame({"Año": all_years_kpi})
    for nm in NAMES:
        for k in kpi_names:
            kdf[f"{k}__{nm}"] = kdf["Año"].map(EMPS[nm]["kpis"][k].to_dict())
    kpi_path = "/content/kpis_comparados_ABC.csv"
    kdf.to_csv(kpi_path, index=False)

    return rat_path, kpi_path

# ---------- UI (Gradio) ----------
with gr.Blocks(title="Comparador Financiero — Empresas A, B y C") as app:
    gr.Markdown("## 🆚 Comparador Financiero — Empresas A, B y C (addon)\nCarga 2 o 3 archivos Excel con la **misma estructura** usada en tu app original.")

    with gr.Row():
        a_file = gr.File(label="Excel Empresa A (.xlsx)")
        b_file = gr.File(label="Excel Empresa B (.xlsx)")
        c_file = gr.File(label="Excel Empresa C (.xlsx) — opcional")
    load_btn = gr.Button("Cargar y calcular")

    # Estados
    EMPS = gr.State()    # dict nombre -> métricas
    NAMES = gr.State()   # lista de nombres (A, B, [C])
    YEARS = gr.State()   # años unión (ratios)
    gr.Markdown("---")

    # --- Tabs comparativas ---
    with gr.Tab("1) KPIs comparados"):
        kpi_dd = gr.Dropdown(choices=['Ventas','Utilidad Neta','Total Activos','Total Pasivos','Patrimonio'], label="KPI")
        kpi_plot = gr.Plot()
        kpi_tbl  = gr.Dataframe(label="Tabla combinada", interactive=False)
        btn_update_kpi = gr.Button("Actualizar KPI")

    with gr.Tab("2) Ratios comparados"):
        ratio_dd = gr.Dropdown(label="Ratio")
        ratio_plot = gr.Plot()
        ratio_tbl  = gr.Dataframe(label="Tabla combinada", interactive=False)
        btn_update_ratio = gr.Button("Actualizar Ratio")

    with gr.Tab("3) Horizontal comparado"):
        gr.Markdown("### Balance General — Horizontal (YoY)")
        bal_h_tbl = gr.Dataframe(interactive=False)
        gr.Markdown("### Estado de Resultados — Horizontal (YoY)")
        er_h_tbl  = gr.Dataframe(interactive=False)
        btn_show_horiz = gr.Button("Ver Horizontales")

    with gr.Tab("4) Vertical por año"):
        empresa_sel_dd = gr.Dropdown(label="Empresa")
        year_dd = gr.Dropdown(label="Año")
        with gr.Row():
            vbal_plot = gr.Plot(label="Balance — % por cuenta (Top 10)")
            ver_plot  = gr.Plot(label="E. Resultados — % por cuenta (Top 10)")
        gr.Markdown("**Tablas por empresa (si existen)**")
        vbal_tblA = gr.Dataframe(label="Balance Vertical — Empresa 1", interactive=False)
        vbal_tblB = gr.Dataframe(label="Balance Vertical — Empresa 2", interactive=False)
        vbal_tblC = gr.Dataframe(label="Balance Vertical — Empresa 3", interactive=False)
        ver_tblA  = gr.Dataframe(label="ER Vertical — Empresa 1", interactive=False)
        ver_tblB  = gr.Dataframe(label="ER Vertical — Empresa 2", interactive=False)
        ver_tblC  = gr.Dataframe(label="ER Vertical — Empresa 3", interactive=False)
        btn_show_vertical = gr.Button("Ver Vertical (año)")

    with gr.Tab("5) Export"):
        comb_ratios = gr.File(label="Ratios combinados (CSV)")
        comb_kpis   = gr.File(label="KPIs combinados (CSV)")
        btn_export = gr.Button("Generar CSVs combinados")

    # --- Enlaces de eventos ---
    load_btn.click(
        _load,
        inputs=[a_file, b_file, c_file],
        outputs=[EMPS, NAMES, YEARS, ratio_dd, year_dd, empresa_sel_dd]
    )

    btn_update_kpi.click(
        _plot_kpi_multi,
        inputs=[kpi_dd, EMPS, YEARS, NAMES],
        outputs=[kpi_plot, kpi_tbl]
    )

    btn_update_ratio.click(
        _plot_ratio_multi,
        inputs=[ratio_dd, EMPS, NAMES],
        outputs=[ratio_plot, ratio_tbl]
    )

    btn_show_horiz.click(
        _horizontal_tables_multi,
        inputs=[EMPS, NAMES],
        outputs=[bal_h_tbl, er_h_tbl]
    )

    btn_show_vertical.click(
        _vertical_by_year_select,
        inputs=[empresa_sel_dd, year_dd, EMPS, NAMES],
        outputs=[vbal_plot, ver_plot, vbal_tblA, vbal_tblB, vbal_tblC, ver_tblA, ver_tblB, ver_tblC]
    )

    btn_export.click(
        _exports_multi,
        inputs=[EMPS, NAMES],
        outputs=[comb_ratios, comb_kpis]
    )

app.queue().launch(share=True, inline=False, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0426d0db4715b9633a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
